In [45]:
import pandas as pd
import numpy as np

In [46]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import Lasso, LogisticRegression
from matplotlib.patches import Patch
import matplotlib.pyplot as plt

In [47]:
df = pd.read_csv("../csv_data/부실징후기업_v1.csv", thousands=",", encoding="CP949")
df.drop(columns=["Unnamed: 0"], inplace=True)

In [48]:
df.sort_values(by="회계년도", ascending=True)

,회사명,거래소코드,회계년도,이자보상배율,cashflow_매출액비율,cashflow_부채비율,당좌비율,매입채무회전율,매출액순이익률,매출액총이익률,...,타인자본회전율,재고자산_유동자산비율,재고자산증가율,재고자산회전율,차입금의존도,총자본영업이익률,총자산순이익률,총자산증가율,총자산회전율,현금비율
0,(주)CMG제약,58820,2011,3.530000,-24.06614599,-46.081054,165.032796,11.982404,-8.496547712,41.832069,...,1.914767,17.970021,0.000000,6.471395,1.750645,1.729531,-5.523791,0.891189,0.650122,15.097437
4145,(주)홈캐스트,64240,2011,11.000000,3.93052326,18.173411,173.981208,5.815343,5.095883059,21.993339,...,2.214855,16.166901,-19.864453,9.542338,11.118305,9.811465,8.297802,-2.323083,1.250924,74.589183
5621,우리바이오(주),82850,2011,2.250000,3.366599766,6.118629,143.673856,107.997835,2.014510068,16.066542,...,1.437246,55.552324,-90.897054,3.314719,17.821940,7.285934,1.662528,-43.537766,0.867898,22.724504
2183,(주)엠피대산,65150,2011,5.010000,8.778131495,21.602517,47.646198,10.203748,2.457718913,36.713719,...,1.353564,19.395134,-0.371470,3.900035,1.260360,16.158859,4.551612,1.031179,0.595181,4.502423
4154,(주)화신,10690,2011,18.360219,5.613060219,15.944302,109.608663,15.412946,2.545873898,9.078323,...,3.390704,53.646544,-71.769335,8.102425,17.062069,9.824093,4.273312,-50.902209,1.348326,32.013966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,(주)에스폴리텍,50760,2019,14.690000,7.90244625,20.084957,106.037995,34.977969,7.949617838,20.276088,...,2.323444,18.528700,-1.158007,3.153191,27.334625,13.248624,9.807002,2.367242,0.562672,53.773401
4391,금호전기(주),1210,2019,-0.568430,-8.522049864,-9.536984,39.949884,25.300969,-52.81044419,26.976557,...,3.710133,56.500766,-37.653212,17.678611,0.549278,-12.389519,-45.473782,-21.612819,1.739162,16.848229
5887,자화전자(주),33240,2019,-13.750000,8.162813693,42.422101,220.250460,23.357818,-0.077367409,11.452859,...,2.459408,19.217083,-46.696710,6.590928,7.446427,-1.643724,-0.080603,-3.805183,0.986459,101.248803
4427,남광토건(주),1260,2019,8.848670,0.671909698,1.060107,101.020824,5.409318,1.635999094,9.499218,...,0.890449,0.089692,0.000000,23.768273,12.462569,4.559055,1.907469,-0.067252,0.824967,29.266924


In [49]:
XX = np.arange(100)
n_split = 5
tscv = TimeSeriesSplit()
print(tscv)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)


In [50]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])

for train_index, test_index in tscv.split(X):
    print("TRAIN: ", train_index, "TEST: ", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN:  [0] TEST:  [1]
TRAIN:  [0 1] TEST:  [2]
TRAIN:  [0 1 2] TEST:  [3]
TRAIN:  [0 1 2 3] TEST:  [4]
TRAIN:  [0 1 2 3 4] TEST:  [5]


In [51]:
df.sort_values(by="회계년도", ascending=True, inplace=True)
df.reset_index(inplace=True, drop=True)
df.head(3)

,회사명,거래소코드,회계년도,이자보상배율,cashflow_매출액비율,cashflow_부채비율,당좌비율,매입채무회전율,매출액순이익률,매출액총이익률,...,타인자본회전율,재고자산_유동자산비율,재고자산증가율,재고자산회전율,차입금의존도,총자본영업이익률,총자산순이익률,총자산증가율,총자산회전율,현금비율
0,(주)CMG제약,58820,2011,3.53,-24.06614599,-46.081054,165.032796,11.982404,-8.496547712,41.832069,...,1.914767,17.970021,0.000000,6.471395,1.750645,1.729531,-5.523791,0.891189,0.650122,15.097437
1,(주)홈캐스트,64240,2011,11.00,3.93052326,18.173411,173.981208,5.815343,5.095883059,21.993339,...,2.214855,16.166901,-19.864453,9.542338,11.118305,9.811465,8.297802,-2.323083,1.250924,74.589183
2,우리바이오(주),82850,2011,2.25,3.366599766,6.118629,143.673856,107.997835,2.014510068,16.066542,...,1.437246,55.552324,-90.897054,3.314719,17.821940,7.285934,1.662528,-43.537766,0.867898,22.724504


In [52]:
df['회계년도'].value_counts()

# 2011 ~ 2016 : 4238
# 2017 ~ 2019 : 2417

2018    814
2019    813
2017    790
2016    777
2015    757
2014    711
2013    680
2012    661
2011    652
Name: 회계년도, dtype: int64

In [53]:
tscv_test = TimeSeriesSplit(n_splits=5)
tscv_test

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)

In [54]:
# X = df[["당좌비율", "현금비율"]][:4238]
# y = df["이자보상배율"][:4238]

In [55]:
# for train_index, test_index in tscv.split(X):
#     print("TRAIN: ", train_index, "TEST: ", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [56]:
# tscv_test_2 = TimeSeriesSplit(n_splits=3)

# XX = df[["당좌비율","현금비율"]][:1000], df[["당좌비율","현금비율"]][1000:2000], df[["당좌비율","현금비율"]][2000:4237]
# yy = df["이자보상배율"][:1000], df["이자보상배율"][1000:2000], df["이자보상배율"][2000:4237]

In [57]:
# XX = np.asarray(XX)
# yy = np.asarray(yy)

# for train_index, test_index in tscv_test_2.split(XX):
#     print("TRAIN: ", train_index, "TEST: ", test_index)
#     XX_train, XX_test = XX[train_index], XX[test_index]
#     yy_train, yy_test = yy[train_index], yy[test_index]

In [58]:
# TimeSeriesSplit을 해보고 싶어서 0, 1 이진 데이터 끌고와서 멋대로 합침

df_binary = pd.read_csv("./checking3.csv", encoding="CP949")
df_binary = df_binary["부실기업"]

In [59]:
df = pd.concat([df, df_binary], axis=1)
df.dropna(inplace=True)

In [60]:
df

,회사명,거래소코드,회계년도,이자보상배율,cashflow_매출액비율,cashflow_부채비율,당좌비율,매입채무회전율,매출액순이익률,매출액총이익률,...,재고자산_유동자산비율,재고자산증가율,재고자산회전율,차입금의존도,총자본영업이익률,총자산순이익률,총자산증가율,총자산회전율,현금비율,부실기업
0,(주)CMG제약,58820.0,2011.0,3.530000,-24.06614599,-46.081054,165.032796,11.982404,-8.496547712,41.832069,...,17.970021,0.000000,6.471395,1.750645,1.729531,-5.523791,0.891189,0.650122,15.097437,0.0
1,(주)홈캐스트,64240.0,2011.0,11.000000,3.93052326,18.173411,173.981208,5.815343,5.095883059,21.993339,...,16.166901,-19.864453,9.542338,11.118305,9.811465,8.297802,-2.323083,1.250924,74.589183,0.0
2,우리바이오(주),82850.0,2011.0,2.250000,3.366599766,6.118629,143.673856,107.997835,2.014510068,16.066542,...,55.552324,-90.897054,3.314719,17.821940,7.285934,1.662528,-43.537766,0.867898,22.724504,1.0
3,(주)엠피대산,65150.0,2011.0,5.010000,8.778131495,21.602517,47.646198,10.203748,2.457718913,36.713719,...,19.395134,-0.371470,3.900035,1.260360,16.158859,4.551612,1.031179,0.595181,4.502423,0.0
4,(주)화신,10690.0,2011.0,18.360219,5.613060219,15.944302,109.608663,15.412946,2.545873898,9.078323,...,53.646544,-71.769335,8.102425,17.062069,9.824093,4.273312,-50.902209,1.348326,32.013966,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6650,(주)에스폴리텍,50760.0,2019.0,14.690000,7.90244625,20.084957,106.037995,34.977969,7.949617838,20.276088,...,18.528700,-1.158007,3.153191,27.334625,13.248624,9.807002,2.367242,0.562672,53.773401,0.0
6651,금호전기(주),1210.0,2019.0,-0.568430,-8.522049864,-9.536984,39.949884,25.300969,-52.81044419,26.976557,...,56.500766,-37.653212,17.678611,0.549278,-12.389519,-45.473782,-21.612819,1.739162,16.848229,1.0
6652,자화전자(주),33240.0,2019.0,-13.750000,8.162813693,42.422101,220.250460,23.357818,-0.077367409,11.452859,...,19.217083,-46.696710,6.590928,7.446427,-1.643724,-0.080603,-3.805183,0.986459,101.248803,0.0
6653,남광토건(주),1260.0,2019.0,8.848670,0.671909698,1.060107,101.020824,5.409318,1.635999094,9.499218,...,0.089692,0.000000,23.768273,12.462569,4.559055,1.907469,-0.067252,0.824967,29.266924,0.0


In [61]:
X = df[["당좌비율", "현금비율"]][:4238]
y = df["부실기업"][:4238]
X = X.fillna(0) # 결측치 있으면 안 돌아가요

tss = TimeSeriesSplit(n_splits=3)
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [75]:
model = Lasso()
#model = LogisticRegression(fit_intercept=True)
#tss_2 = TimeSeriesSplit(n_splits=3)
scores = cross_val_score(model, X, y, cv=tss, scoring="neg_mean_squared_error")

In [81]:
np.mean(scores)*-1

0.13900655959916805